# Constructing the magnetic diffusion

In [11]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sympy as sym
from pg_utils.pg_model import core, forcing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Displaying utilities

In [5]:
def display_collection(collection, _filter=lambda item, idx: True):
    display(*(item for idx, item in enumerate(collection) if _filter(item, idx)))

# Linear drag model

In [13]:
display_collection(forcing.Dm_models_cg_lin["linear drag"])

None

-\overline{m_1}(s, \phi, t)

-\overline{m_+}(s, \phi, t)

-\overline{m_-}(s, \phi, t)

-\widetilde{m_{z+}}(s, \phi, t)

...

-b_-^+(s, \phi, t)

-b_z^+(s, \phi, t)

-b_+^-(s, \phi, t)

-b_-^-(s, \phi, t)

-b_z^-(s, \phi, t)

In [19]:
import json
from pg_utils.pg_model import base
with open('./out/eigen/Toroidal_Quadrupole/eqs_cg.json', 'r') as fread:
    load_array = json.load(fread)
    eqs = base.LabeledCollection.deserialize(load_array['equations'], parser=sym.parse_expr)
    pars = [sym.parse_expr(par) for par in load_array["params"]]
with open('./out/eigen/Toroidal_Quadrupole/eqs_cg_drag.json', 'r') as fread:
    load_array = json.load(fread)
    eqs2 = base.LabeledCollection.deserialize(load_array['equations'], parser=sym.parse_expr)
    pars2 = [sym.parse_expr(par) for par in load_array["params"]]

for fname in eqs._field_names:
    if eqs[fname] is not None:
        print(
            fname, 
            eqs[fname].lhs.equals(eqs2[fname].lhs),
            eqs[fname].rhs.equals(eqs2[fname].rhs)
        )
        display(eqs2[fname].rhs - eqs[fname].rhs)

Psi True True


0

M_1 True False


-\overline{m_1}(s, \phi, t)/\mathrm{Lu}

M_p True False


...

-b_+^-(s, \phi, t)/\mathrm{Lu}

B_mm True False


-b_-^-(s, \phi, t)/\mathrm{Lu}

Bz_m True False


-b_z^-(s, \phi, t)/\mathrm{Lu}